In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('/content/drive/MyDrive/weit/seoul/서울민원데이터.csv')

In [8]:
!pip install folium

In [9]:
import folium
from folium import Marker

In [ ]:
import requests
import json

# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11,
    tiles='cartodbpositron'
)


In [ ]:
seoul_group_data = data.groupby('City')['Dong'].count()
seoul_group_data

In [ ]:
folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

m.choropleth(geo_data=seoul_geo,
             data=seoul_group_data,
             fill_color='YlOrRd',
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name="서울시 구별 민원 수"
            )
m

In [ ]:
m.save('map.html')

# 동별 시각화

In [ ]:
jpath = '/content/drive/MyDrive/weit/seoul/Dong.geojson'
seoul_geo = json.load(open(jpath, encoding='utf-8'))

In [ ]:
data['Location'] = data['Province'] + ' ' + data['City'] + ' ' + data['Dong']

In [ ]:
seoul_group_data = data.groupby('Location')['Dong'].count()
seoul_group_data

Location
서울특별시 강남구 개포동    190
서울특별시 강남구 논현동    327
서울특별시 강남구 대치동    396
서울특별시 강남구 도곡동    165
서울특별시 강남구 삼성동    485
                ... 
서울특별시 중랑구 면목동    419
서울특별시 중랑구 묵동      85
서울특별시 중랑구 상봉동    147
서울특별시 중랑구 신내동    279
서울특별시 중랑구 중화동     56
Name: Dong, Length: 436, dtype: int64

In [ ]:
g_m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11,
    tiles='openstreetmap'
)

In [ ]:
fmap = folium.Choropleth(geo_data=seoul_geo,
              data=seoul_group_data,
              fill_color='YlOrRd',
              fill_opacity=0.5,
              line_opacity=1,
              key_on='feature.properties.adm_nm',
              legend_name="서울시 동별 민원 수"
              ).add_to(g_m)


In [ ]:
folium.LayerControl().add_to(g_m)


In [ ]:
fmap.geojson.add_child(
    folium.features.GeoJsonTooltip(['adm_nm'],labels=False)
)
fmap.geojson.zoom_on_click = False

In [ ]:
g_m

In [ ]:
filtered_df = data[data['Dong'] == '왕십리동']

# 결과 확인
print(filtered_df)

Empty DataFrame
Columns: [발생일, 위도, 경도, CODE, Province, City, Dong, Location]
Index: []


# 전국 시각화

In [ ]:
import requests
import json

# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-provinces-2018-geo.json')
c = r.content
seoul_geo = json.loads(c)

In [ ]:
uturn = pd.read_csv('/content/drive/MyDrive/weit/불법유턴.csv')
#leftturn=pd.read_excel('/content/drive/MyDrive/weit/불법좌회전.xlsx')
#signalviolation=pd.read_excel('/content/drive/MyDrive/weit/신호위반.xlsx')
stopline=pd.read_csv('/content/drive/MyDrive/weit/정지선침범.csv')
centerline=pd.read_csv('/content/drive/MyDrive/weit/중앙선침범.csv')
#crackdown=pd.read_excel('/content/drive/MyDrive/weit/진로변경방법위반.xlsx')

In [ ]:
def address_to_dataframe(df):

    addresses = df['ADDRESS']

    provinces = []


    for address in addresses:
        address_str = str(address)  # Ensure the value is a string
        parts = address_str.split()
        provinces.append(parts[0])


    df['Province'] = provinces

    return df

In [ ]:
Filteruturn= address_to_dataframe(uturn)
Filterstopline= address_to_dataframe(stopline)
Filtercenterline= address_to_dataframe(centerline)


In [ ]:
Filteruturn

,Unnamed: 0,발생일,위도,경도,ADDRESS,CODE,Province
0,0,2022-12-31,34.982894,127.578694,전라남도 광양시 광양읍 구산리,4.623025e+09,전라남도
1,1,2022-12-31,35.817600,128.533500,대구광역시 달서구 상인동,2.729012e+09,대구광역시
2,2,2022-12-31,34.972354,127.589357,전라남도 광양시 광양읍 목성리,4.623025e+09,전라남도
3,3,2022-12-31,36.113998,128.363566,경상북도 구미시 신평동,4.719011e+09,경상북도
4,4,2022-12-31,35.178612,126.895306,광주광역시 북구 용봉동,2.917011e+09,광주광역시
...,...,...,...,...,...,...,...
15391,15391,2022-01-06,37.331868,127.093384,경기도 용인시 수지구 풍덕천동,4.146510e+09,경기도
15392,15392,2022-01-05,36.636857,127.459641,충청북도 청주시 서원구 사창동,4.311210e+09,충청북도
15393,15393,2022-01-05,35.184418,128.990568,부산광역시 사상구 모라동,2.653010e+09,부산광역시
15394,15394,2022-01-05,37.631195,127.117372,경기도 구리시 갈매동,4.131010e+09,경기도


In [ ]:
global_dict = json.loads(requests.get('https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-provinces-2018-geo.json').text)

In [ ]:
group_data = Filtercenterline.groupby('Province')['Province'].count()
data = {
    'Province': group_data.index,
    'Count': group_data.values
}

# Create a new DataFrame
new_df = pd.DataFrame(data)

# If you want to exclude the row with 'nan', you can do this:
new_df = new_df[new_df['Province'] != 'nan']

# Reset the index of the new DataFrame
new_df.reset_index(drop=True, inplace=True)
new_df['Province'] = new_df['Province'].replace('강원특별자치도', '강원도')

In [ ]:
new_df

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11,

)


In [ ]:
folium.Choropleth(
    geo_data=global_dict,
    data=new_df,
    columns=['Province','Count'],
    key_on='properties.name',
    fill_color='YlOrRd',
).add_to(m)
m

In [ ]:
m.save('centerline.html')

# 원하는 동만 출력

In [53]:
data_1 = data[data['Dong']=='가산동']
data_1 = data_1.reset_index(drop=True)
latitude, longitude, date= data_1['위도'], data_1['경도'], data_1['발생일']

In [54]:
from folium.plugins import MiniMap
m = folium.Map(location=[latitude[0], longitude[0]], zoom_start=10)

for i in range(len(data_1)):
  folium.Marker([latitude[i], longitude[i]],
  popup=date[i],
  icon=folium.Icon(color='red', icon='star')).add_to(m)

# Create a MiniMap
#minimap = MiniMap()
#minimap.add_to(m)

# Add a legend to the main map
#folium.Marker([latitude[0], longitude[0]], icon=folium.DivIcon(html='<div>상암동 민원</div>')).add_to(m)

#m


In [27]:
m.save('상암동.html')

# CCTV

In [30]:
shin_cctv=pd.read_csv('/content/drive/MyDrive/weit/seoul/서울특별시_관악구_무인교통단속카메라.csv',encoding='cp949')
sang_cctv=pd.read_csv('/content/drive/MyDrive/weit/seoul/서울특별시_마포구_무인교통단속카메라.csv',encoding='cp949')
ga_cctv=pd.read_csv('/content/drive/MyDrive/weit/seoul/서울특별시_금천구_무인교통단속카메라.csv',encoding='cp949')

In [36]:
def clear(df):
   selected_columns = ['위도', '경도', '소재지지번주소']
   df = df[selected_columns]
   df = df.reset_index(drop=True)
   return df

In [55]:
cctv = clear(ga_cctv)
cctv_lat, cctv_lot, = cctv['위도'], cctv['경도']

In [56]:
 # https://guideman.tistory.com/21

for i in range(len(cctv)):
  folium.Marker([cctv_lat[i], cctv_lot[i]],
    icon=folium.Icon(color='cadetblue', icon='star')).add_to(m)

  folium.CircleMarker([cctv_lat[i], cctv_lot[i]],
    radius=40,
    color='green',
    fill_color='green').add_to(m)



In [ ]:
m

In [57]:
m.save('가산동cctv.html')